<h1 align=center><font size=5>NYC Traffic Open Data: Planning Proposal Simulator</font></h1>
<hr>
<i>This is a semester-long project for the course <b>ARTS-UG 1647 Making Virtual Sense</b> at <b>New York University, Gallatin School of Individualized Study</b></i>.

<i>--- Developed under the advice of <b>Doctor Carl Skelton</b>, solely by <b>Michael Xu</b> (NYU NetID: tx542; Full Legal Name: Tianxiao Xu)</i>

<i>Usage of this project end product demonstration ("demo") is contingent upon the terms and conditions listed in the <a href="https://github.com/tx542/nyc_open_data/blob/main/Project%20Notice%20of%20Objectives%20and%20Limitations.ipynb" target="_blank">"Notice of Project Objectives and Limitations"</a> document.</i>
<hr>
<h4>Instructions:</h4>
You have <b>four</b> possible modification options (different scenarios) to run the simulation with:

<ol>
  <li><b>None: No modifications.</b> You can <b>select a borough</b> in the <i>Borough</i> section to view a descriptive 
  interactive illustration (on the map below) of the traffic network patterns of the <i>entire borough</i>. Note: 
  viewing the traffic information and metadata of a specific street has been depreciated, due to the interdependent and 
  inter-correlated nature of traffic networks on the borough level. </li>
  <li><b>Construction: Place a specific road under construction.</b> You can select a borough in the <i>Borough</i> section, 
  and the name of a specific street (with real-time surveillance data available from the 
  <i><u><a href = "https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq" target="_blank" style="color:blue;">
  NYC Open Data database</a></u></i>) of that borough in the <i>Traffic Link ...</i> section to place it under construction. 
  And as this road will be under construction, it will be taken down from the overall
  traffic network, thereby (likely) influencing the borough-level traffic patterns. </li>
  <li><b>New Road: Proposal to build a new road (in addition to the existing "grid" traffic network).</b> 
  You will <b>only</b> need to enter the street address (as accurate as possible) of the <b>starting point</b> in the 
  <i>Starting Point</i> section, and that of the <b>ending point</b> in the <i>Ending Point</i> section. 
  You won't need to enter anything else in any other sections (they will be invisible anyway, to avoid confusion).</li>
  <li><b>New Multi-Road: Proposal to build a new road, with multiple stopping points.</b> In the <i>Multi-Road</i>
  text box, enter the street address of each of the proposed stopping points in a new line (separate by the <i>enter/return</i> key). 
  Make sure that the address for every location is <b>accurate (to the best of your knowledge)</b>—otherwise,
  the simulator may not be able to locate your proposed point and return an error. </li>
</ol>

<p style="color:blue;"><u><a href="https://github.com/tx542/nyc_open_data/blob/main/Project%20Notice%20of%20Objectives%20and%20Limitations.ipynb" target="_blank">Read <i>Notice of Project Objectives and Limitations</i></a></u></p>
<hr>

In [1]:
import os
import os.path
from os import path
import time
import datetime
import pytz

import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, LineString
# import geoplot as gplt

import plotnine
from plotnine import *
# set the plotnine figure size
plotnine.options.figure_size = (8, 6)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%matplotlib widget

from sodapy import Socrata
import json
import matplotlib.pyplot as plt
import folium
from IPython.core.display import display, HTML
import ipywidgets as wg

import warnings
warnings.filterwarnings('ignore')

# predictive model libraries:

import statsmodels.formula.api as smf
from sklearn.neighbors import KNeighborsRegressor as knn
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split, cross_val_score

# PCA - Dimension Reduction
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import DistanceMetric
from sklearn.neighbors import BallTree
# All valid metrics:
# BallTree.valid_metrics

# Nominatim requests
import requests
import urllib.parse

DATASET_ID = {
"CarCrashes": "h9gi-nx95",
"LiveTraffic": "i4gi-tjb9"
}

FIG_WIDTH = 16
FIG_HEIGHT = 9

# sets timezone 
os.environ['TZ'] = "America/New_York"
time.tzset()

# import my own project files 
from Open_Data import * # class definition
from all_functions import * # helper functions
from reg_func import * # regressor optimization functions

# --->>> object instantiation <<<---

# MyCrashes = Crashes("CarCrashes", 10000)
# MyCrashes.static_map()

MyTraffic = RTraffic("LiveTraffic", 1000)
MyTraffic.get_polylines()
# MyTraffic.display_folium()

all_links = list(MyTraffic.pol_df["names"].unique())
all_boros = ["Bronx", "Brooklyn", "Manhattan", "Queens", "Staten Island"]

In [3]:
# make street names and boroughs categorical data
# and extract categorical code (for predictions)

MyTraffic.pol_df.names = pd.Categorical(MyTraffic.pol_df.names)
MyTraffic.pol_df['names_code'] = MyTraffic.pol_df.names.cat.codes

MyTraffic.pol_df.boro = pd.Categorical(MyTraffic.pol_df.boro)
MyTraffic.pol_df['boro_code'] = MyTraffic.pol_df.boro.cat.codes


# make the speed and time col vars numeric (apply float)

MyTraffic.pol_df["rou_speed"] = MyTraffic.pol_df["rou_speed"].apply(float)
MyTraffic.pol_df["tr_time"] = MyTraffic.pol_df["tr_time"].apply(float)


# convert time (timestamp) to a np.float64 object

MyTraffic.pol_df["time_float"] = MyTraffic.pol_df["time_s"].apply(lambda x: np.datetime64(x).astype("float"))

# NOTE: this_time = np.datetime64('2018-04-01T15:30:00').astype("float")
# revert_date = np.datetime64(datetime.datetime.utcfromtimestamp(this_time))

# Alt: convert time (timestamp) to a Pandas Datetime object
# MyTraffic.pol_df["time_s"] = pd.to_datetime(MyTraffic.pol_df["time_s"], format = "%Y-%m-%dT%H:%M:%S")

# how to parse string into datetime object:
# https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes


# calculate the distance (travel distance) for each recorded street (one end point to another)

MyTraffic.pol_df["tr_dist"] = MyTraffic.pol_df["geometry"].apply(lambda x: float(x.length))


# drop all recorded speed of 0 mile per hour or a travel time of 0 seconds
# (invalid observation - based on realistic life intuition)
# WHAT IS MISSING (EXCLUDED DUE TO INVALID OBS)

MyTraffic.pol_df = MyTraffic.pol_df[MyTraffic.pol_df["rou_speed"] != 0]
MyTraffic.pol_df = MyTraffic.pol_df[MyTraffic.pol_df["tr_time"] != 0]

In [3]:
# KNN Second model (trying to predict travel time)

opt_knn_fit_model_2 = knn(n_neighbors = 7).fit(X = MyTraffic.pol_df[['names_code', 'boro_code', 
                                                                     'time_float', 'tr_dist']],
y = MyTraffic.pol_df['tr_time'])

# create the column of predicted KNN values
MyTraffic.pol_df['opt_pred_knn_2'] = opt_knn_fit_model_2.predict(MyTraffic.pol_df[['names_code', 'boro_code',
                                                                                   'time_float', 'tr_dist']])

# obtain score of fit
# opt_knn_fit_model_2.score(X = MyTraffic.pol_df[['names_code', 'boro_code', 'time_float', 'tr_dist']], 
#                   y = MyTraffic.pol_df['tr_time'])

In [4]:
# KNN First model (trying to predict travel speed)

# if actual data is used (df = 1)
opt_knn_fit_model_1 = knn(n_neighbors = 7).fit(X = MyTraffic.pol_df[['names_code', 'boro_code', 
                                                                   'time_float', 'tr_time', 'tr_dist']],
y = MyTraffic.pol_df['rou_speed'])

# create the column of predicted KNN values
MyTraffic.pol_df['opt_pred_knn_1'] = opt_knn_fit_model_1.predict(MyTraffic.pol_df[['names_code', 'boro_code', 
                                                                   'time_float', 'tr_time', 'tr_dist']])

# obtain score of fit
# opt_knn_fit_model_1.score(X = MyTraffic.pol_df[['names_code', 'boro_code', 'time_float', 'tr_time', 'tr_dist']], 
#                   y = MyTraffic.pol_df['rou_speed'])

In [5]:
# kv maps to correspond categorical codes with labels 

MyTraffic.df_names_kv = dict(zip(MyTraffic.pol_df.names.unique().tolist(),
                                 MyTraffic.pol_df['names_code'].unique().tolist()))

MyTraffic.df_names_kv_rev = dict(zip(MyTraffic.pol_df['names_code'].unique().tolist(),
                                     MyTraffic.pol_df.names.unique().tolist()))

MyTraffic.df_boro_kv = dict(zip(MyTraffic.pol_df.boro.unique().tolist(),
                                MyTraffic.pol_df['boro_code'].unique().tolist()))


In [6]:
def make_pred(usr_time, some_df):
    # convert datetime obj to str and float
    time_float = np.datetime64(usr_time).astype("float")
    time_str = usr_time.strftime("%Y-%m-%dT%H:%M:%S")
    
    # add value to df cols 
    some_df['time_float'] = time_float // 1000
    some_df['time_s'] = time_str
    
    # first predictor object, to derive travel time
    # [2nd optimal KNN model prediction (to derive travel time)]
    some_df['pred_tr_time'] = opt_knn_fit_model_2.predict(some_df[['names_code', 'boro_code',
                                                                   'time_float', 'tr_dist']])
    
    # second predictor object, to derive travel speed
    # [1st optimal KNN model prediction (to derive travel speed)]
    some_df['pred_rou_speed'] = opt_knn_fit_model_1.predict(some_df[['names_code', 'boro_code',
                                                                     'time_float', 'pred_tr_time', 'tr_dist']])
    

In [7]:
def get_pop_estimate(usr_str):
    
    '''
    
    This function uses the Population Estimation Service (PES) [Gridded 
    Population of the World (GPW), v4] to derive the estimated population
    within the boundary coordinates of the address entered by the user
    in string format. 
    
    --- >>> References and Data Sources <<< ---
    
    OpenStreetMap (OSM) - Nominatim
    (c) OpenStreetMap contributors
    
    OpenStreetMap® is open data, licensed under the Open Data Commons Open Database License (ODbL) 
    by the OpenStreetMap Foundation (OSMF). The data is available under the Open Database License.
    https://www.openstreetmap.org/copyright
    
    ---
    
    Center for International Earth Science Information Network - CIESIN - Columbia University. 
    2018. Population Estimation Service, Version 3 (PES-v3). Palisades, NY: NASA Socioeconomic 
    Data and Applications Center (SEDAC). https://doi.org/10.7927/H4DR2SK5. Accessed DAY MONTH YEAR.
    
    https://sedac.ciesin.columbia.edu/data/collection/gpw-v4/population-estimation-service
    
    '''
    
    # URL request to Nominatim (Open Street Map)
    url = 'https://nominatim.openstreetmap.org/search/' +\
    urllib.parse.quote(usr_str) +\
    '?format=json'

    # get the first (most relevant) returned search result
    response = requests.get(url).json(strict = False)[0]
    osm_obj = response
    
    # -----------------------------------------------------------
    # min latitude, max latitude, min longitude, max longitude
    bbox = [float(item) for item in response['boundingbox']]
    
    # drawing the polygon around the input address
    add_polygon = [[bbox[2],bbox[0]], [bbox[3],bbox[0]],
    [bbox[3],bbox[1]], [bbox[2],bbox[1]], [bbox[2],bbox[0]]]
    
    # verify area size of boundary around input address
    temp_loc_x = [item[0] for item in add_polygon]
    temp_loc_y = [item[1] for item in add_polygon]

    temp_loc_pts = gpd.points_from_xy(temp_loc_x, 
                                      temp_loc_y, 
                                      crs = "EPSG:4326")

    temp_loc_geometry = LineString(temp_loc_pts)
    temp_loc_tr_dist = temp_loc_geometry.length
    
    
    # -----------------------------------------------------------
    # a longer periphery, so that data is obtainable
    lbox = bbox
    lbox[0] -= 0.05
    lbox[2] -= 0.05
    lbox[1] += 0.05
    lbox[3] += 0.05
    
    # make the `longer` polygon
    long_polygon = [[lbox[2],lbox[0]], [lbox[3],lbox[0]],
    [lbox[3],lbox[1]], [lbox[2],lbox[1]], [lbox[2],lbox[0]]]
    # print(long_polygon)
    
    # LineString obj for the `longer` area
    
    temp_long_x = [item[0] for item in long_polygon]
    temp_long_y = [item[1] for item in long_polygon]

    temp_long_pts = gpd.points_from_xy(temp_long_x, 
                                       temp_long_y, 
                                       crs = "EPSG:4326")

    temp_long_geometry = LineString(temp_long_pts)
    temp_long_tr_dist = temp_long_geometry.length
    
    # calculate the short-to-long ratio
    # ratio of length, so square it
    loc_long_ratio = (temp_loc_tr_dist / temp_long_tr_dist) ** 1.5
    # print(loc_long_ratio)

    # the server to query the population estimation data(base):
    quest_server = "https://sedac.ciesin.columbia.edu/arcgis/rest/services/sedac/pesv3Broker/GPServer/pesv3Broker/execute?"

    # temporary python dictionary,
    # functioning as form (query) input fields/parameters
    temp = {"Input_Data": {
        "polygon": long_polygon,
        "variables": ["gpw-v4-population-count-rev10_2020",
                     "gpw-v4-land-water-area-rev10_landareakm"],
        "statistics": ["COUNT", "SUM", "MEAN"],
        "requestID": "123456789"}, "f": "pjson"}

    url = quest_server + urllib.parse.urlencode(temp)
    # not strict json parsing, to allow for control characters
    response = requests.get(url).json(strict = False)
    
    
    # -----------------------------------------------------------
    # obtain estimates of `longer` polygon, 
    # extrapolate to derive estimates of the shorter (actual) region
    
    long_pop = int(response['results'][0]['value']['estimates']\
    ['gpw-v4-population-count-rev10_2020']['SUM'])
    long_area = int(response['results'][0]['value']['estimates']\
    ['gpw-v4-land-water-area-rev10_landareakm']['SUM'])
    
    short_pop = loc_long_ratio * long_pop
    short_area = loc_long_ratio * long_area
    
    return {"population(persons)": short_pop,
           "area(km^2)": short_area,
           "OSM_info": osm_obj}


In [141]:
def comp_model2(Mod_label, Modification, time_infotext, u_day, u_hour, u_min, u_sec, t_info, 
                Borough, Link_Name, start_str, end_str, all_address, ethics, ethics_about):

    '''
    Traffic Light Color Scheme (hex code):
    Red: #BB1E10
    Green: #33A532
    Yellow: #F7B500
    '''
    
    if ethics == False:
        display(HTML('''
        <p style="color:red;">Have you read the <u><a href="https://github.com/tx542/nyc_open_data/blob/main/Project%20Notice%20of%20Objectives%20and%20Limitations.ipynb" target="_blank"><i>Notice of Project Objectives and Limitations</i></a></u> 
        yet?<br>You cannot use this Simulator until you have read, understood, and fully 
acknowledged the aforementioned document by checking the checkbox above!</p>
        '''))
        return None
    
    # construct pd.timestamp object
    usr_t_str = "{}T{:02d}:{:02d}:{:02d}".format(u_day, u_hour, u_min, u_sec)
    
    Times = pytz.timezone('America/New_York').localize(pd.to_datetime(usr_t_str))
    
    # Modification:
    # options = ["None", "Construction", "New Road", "New Multi-Road"]

    # kv for focus of map (depends on borough)
    focus_of_map = {"Bronx": [40.8448, -73.8648],
                    "Brooklyn": [40.6782, -73.9442],
                    "Manhattan": [40.7831, -73.9712],
                    "Queens": [40.7282, -73.7949],
                    "Staten Island": [40.5795, -74.1502]
                   }

    # define temporary df to operate on
    temp_exp_df = MyTraffic.pol_df.copy()
    temp_exp_df = temp_exp_df[["names", 'names_code', 'boro', 'boro_code', 'geometry', 'tr_dist']].drop_duplicates()


    # ------>>>>>> Function Based on User's Choice of Modification <<<<<<------

    if Modification == "New Road":

        # try to get coordinates from user inputs
        try:
            start_pt = get_coor(start_str)
            end_pt = get_coor(end_str)
        except:
            print("Invalid starting/ending address!")
            return None

        # if can be found:
        # WE NEED:
        # - 'names_code' (based on proximity)
        # - 'boro_code' (based on address)
        # - 'tr_dist' (can be calculated based on start & end pts)
        # - 'time_float' (user input)

        # determine borough:

        st_borough = det_boro(start_pt[2])
        end_borough = det_boro(end_pt[2])

        # for now: start and end should be in the same borough

        if st_borough != end_borough:
            # print(st_borough, end_borough, st_borough == end_borough, sep = "\n")
            print("The starting and ending points must be in the same borough!")
            return None

        # name of corresponding borough
        correspond_boro = st_borough

        # determine travel distance
        # and construct Linestring object

        temp_pts = gpd.points_from_xy([start_pt[1], end_pt[1]],
                                      [start_pt[0], end_pt[0]],
                                      crs = "EPSG:4326")

        this_geometry = LineString(temp_pts)
        this_tr_dist = this_geometry.length

        # approximate - to which existing streets this new street
        # behaves most similarly to (KNN)

        # use 'geometry' ?
        this_appr_name_code = knn().fit(X = temp_exp_df[['boro_code', 'tr_dist']],
                                        y = temp_exp_df['names_code']).\
        predict(X = [[MyTraffic.df_boro_kv[correspond_boro], this_tr_dist]])


        # add entry of this new road into dataframe
        temp_exp_df = temp_exp_df.append({"names": "***Proposed New Road***",
                                          'names_code': this_appr_name_code,
                                          'boro': correspond_boro,
                                          'boro_code': MyTraffic.df_boro_kv[correspond_boro],
                                          'geometry': this_geometry,
                                          'tr_dist': this_tr_dist
                                         }
                                         , ignore_index = True)

    elif Modification == "New Multi-Road":

        add_ls = []

        for address_line in all_address.split("\n"):

            # get rid of extra white spaces/blank characters
            address_line = address_line.strip()

            # try to get coordinates from user inputs
            try:
                pt_coord = get_coor(address_line)
            except:
                print("Invalid point address!")
                return None

            add_ls.append(pt_coord)

        # if can be found:
        # WE NEED:
        # - 'names_code' (based on proximity)
        # - 'boro_code' (based on address)
        # - 'tr_dist' (can be calculated based on start & end pts)
        # - 'time_float' (user input)

        # determine borough:

        boro_ls = [det_boro(pt[2]) for pt in add_ls]

        # for now: all points should be in the same borough

        if not all(item == boro_ls[0] for item in boro_ls):
            print("All point addresses must be in the same borough!")
            print(boro_ls)
            return None

        # name of corresponding borough
        correspond_boro = boro_ls[0]

        # determine travel distance
        # and construct Linestring object

        temp_pts = gpd.points_from_xy([item[1] for item in add_ls],
        [item[0] for item in add_ls],
        crs = "EPSG:4326")

        this_geometry = LineString(temp_pts)
        this_tr_dist = this_geometry.length

        # approximate - to which existing streets this new street
        # behaves most similarly to (KNN)

        # use 'geometry' ?
        this_appr_name_code = knn().fit(X = temp_exp_df[['boro_code', 'tr_dist']],
                                        y = temp_exp_df['names_code']).\
        predict(X = [[MyTraffic.df_boro_kv[correspond_boro], this_tr_dist]])


        # add entry of this new road into dataframe
        temp_exp_df = temp_exp_df.append({"names": "***Proposed New Road***",
                                          'names_code': this_appr_name_code,
                                          'boro': correspond_boro,
                                          'boro_code': MyTraffic.df_boro_kv[correspond_boro],
                                          'geometry': this_geometry,
                                          'tr_dist': this_tr_dist
                                         }
                                         , ignore_index = True)


    elif Modification == "Construction":

        # verifies that link name is available
        if Link_Name not in all_links:
            print("Please input your selections...")
            return None

        # if road blocked: drop the cat_code correponding to the name
        road_blocked = Link_Name # TB user input

        correspond_boro = temp_exp_df[temp_exp_df['names_code'] ==\
        MyTraffic.df_names_kv[road_blocked]]['boro'].unique().to_list()[0]

        # make a temporary df for analysis
        # temp_exp_df = temp_exp_df[temp_exp_df['names_code'] != MyTraffic.df_names_kv[road_blocked]]

    else:
        # Modification == "None"

        # verifies that link name is available
        if Link_Name not in all_links:
            print("Please input your selections...")
            return None

        correspond_boro = temp_exp_df[temp_exp_df['names_code'] ==\
        MyTraffic.df_names_kv[Link_Name]]['boro'].unique().to_list()[0]

    # ------>>>>>> Operations Below to Display DF on Map <<<<<<------

    # make the relevant predictions
    make_pred(Times, temp_exp_df)

    # extract sub-df with relevant borough name
    this_df = temp_exp_df[temp_exp_df['boro_code'] == MyTraffic.df_boro_kv[correspond_boro]]

    this_f_map = folium.Map(focus_of_map[correspond_boro],
                            tiles = "Stamen Toner",
                            zoom_start = 12)

    # firstly, add (display) all the linestrings for the corresponding borough

    l_idx = 0

    for l_str in this_df["geometry"]:

        ave_speed = this_df['pred_rou_speed'].apply(float).mean()

        this_speed = float(this_df["pred_rou_speed"].tolist()[l_idx])

        # the modified road here
        if Modification == "Construction":
            if (this_df["names_code"].tolist()[l_idx] == MyTraffic.df_names_kv[road_blocked]):
                this_color = '#F7B500'
            # for other unimpacted roads
            elif this_speed > ave_speed:
                # faster than **average** speed
                this_color = '#33A532'
            else:
                this_color = '#BB1E10'

        elif Modification in ["New Road", "New Multi-Road"]:
            if (this_df["names_code"].tolist()[l_idx] == this_appr_name_code):
                
                this_color = '#F7B500'
                
                # ethical info about this new road proposal
                
                max_count = len(list(l_str.coords))

                f = wg.IntProgress(min = 0, 
                                   max = max_count) # instantiate the bar
                display(f) # display the bar
                f.description = 'Loading:'
                count = 0
                
                for rd_pt in list(l_str.coords):
                    
                    # print([rd_pt[1], rd_pt[0]])
                    
                    # use coordinates to get population estimate
                    temp_pt_res = get_pop_estimate(str(rd_pt[::-1])[1:-1])
                    
                    # progress bar
                    
                    f.description = temp_pt_res['OSM_info']['display_name'].\
                    split(correspond_boro)[0].strip()[:-1]
                    
                    print('Analyzing ethical considerations for the proposed destination \033[1;37;40m"{}"\033[0;0;0m.'.\
                         format(f.description))
                    
                    ethics = '''
                    <center>
                    <p style="color:{html_c};">
                    <b>NEW ROAD</b>
                    </p>
                    </center>
                    <hr>
                    <center>
                    <p><b>Date:</b> {date_rec} <b>|</b> <b>Time:</b> {t_rec}
                    </p>
                    </center>
                    <hr>
                    <p>This address point of the <b>new road</b> that you have proposed is:
                    </p>
                    <ul>
                        <li>Of the full address name: <b>{addr}</b></li>
                        <li>With <b>latitude {lat}</b> and <b>longitude {long}</b></li>
                        <li>Of the OSM class <b>{which_class}</b></li>
                        <li>Of the OSM type <b>{which_type}</b></li>
                        <li>Of the OSM importance <b>{which_imp}</b></li>
                    </ul>
                    According to OpenStreetMap, 
                    Data (c) OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright
                    <hr>
                    <p>
                    Also, a new road construction will negatively impact: 
                    </p>
                    <ul>
                        <li><b>The lives of {ppl:.0f} people</b></li>
                        <li><b>An area of {area:.4f} acres</b></li>
                    </ul>
                    <hr>
                    <p><b>Predicted Speed at This Time:</b> {speed:.2f} mph
                    </p>
                    <p><b>Predicted Travel Time:</b> {tr_t:.2f} minutes.
                    </p>
                    <hr>
                    <p><b>TrafficLink Name:</b> {lk_nm}
                    </p>
                    <p><b>Borough:</b> {bor}
                    </p>
                    '''.format(lk_nm = this_df["names"].tolist()[l_idx],
                       bor = correspond_boro,
                       date_rec = this_df["time_s"].tolist()[l_idx].split("T")[0],
                       t_rec = this_df["time_s"].tolist()[l_idx].split("T")[1],
                       tr_t = float(this_df["pred_tr_time"].tolist()[l_idx]) / 60,
                       speed = this_speed,
                       html_c = this_color,
                        which_class = temp_pt_res['OSM_info']['class'],
                        which_type = temp_pt_res['OSM_info']['type'],
                        which_imp = temp_pt_res['OSM_info']['importance'],
                        ppl = temp_pt_res['population(persons)'],
                        addr = temp_pt_res['OSM_info']['display_name'].\
                               split(correspond_boro)[0].strip()[:-1],
                        lat = temp_pt_res['OSM_info']['lat'],
                        long = temp_pt_res['OSM_info']['lon'],
                        area = temp_pt_res['area(km^2)'])
                    
                    temp_iframe = folium.IFrame(ethics)
                    msg_popup = folium.Popup(temp_iframe,
                                             min_width=300,
                                             max_width=300,
                                             min_height=500,
                                             max_height=500)

                    folium.Marker(
                        location = [rd_pt[1], rd_pt[0]],
                        popup = msg_popup,
                        icon = folium.Icon(color="red", prefix='fa', icon="exclamation-triangle"),
                        tooltip = "About this proposed new road location!"
                    ).add_to(this_f_map)
                    
                    # signal to increment the progress bar
                    time.sleep(.1)
                    count += 1
                    f.value += 1 
                
                print("Ethical analyses complete. Click on warning signs on map to view details.")
                
            # for other unimpacted roads
            elif this_speed > ave_speed:
                # faster than **average** speed
                this_color = '#33A532'
            else:
                this_color = '#BB1E10'

        else:
            if this_speed > ave_speed:
                # faster than **average** speed
                this_color = '#33A532'
            else:
                this_color = '#BB1E10'

        folium.Choropleth(
            l_str,
            line_weight=8,
            line_color=this_color,
            key_on='names'
        ).add_to(this_f_map)

        l_idx += 1

    # then, add the meta-info, in the form of pop-up markers,
    # at the starting point of each recorded road

    row_idx = 0

    # for the starting point in every geometry (recorded road)
    for start_pt in this_df["geometry"].apply(lambda x: list(list(x.coords)[0])[::-1]):

        ave_speed = this_df['pred_rou_speed'].apply(float).mean()

        this_speed = float(this_df["pred_rou_speed"].tolist()[row_idx])

        constr_rd = False
        new_rd = False
        travel_color = ""
        # the modified road here
        if Modification == "Construction":
            if (this_df["names_code"].tolist()[row_idx] == MyTraffic.df_names_kv[road_blocked]):
                this_color = '#F7B500'
                constr_rd = True
                travel_color = "Yellow"

            elif this_speed > ave_speed:
                # faster than **average** speed
                this_color = '#33A532'
                travel_color = "Green"

            else:
                this_color = '#BB1E10'
                travel_color = "Red"

        elif Modification in ["New Road", "New Multi-Road"]:
            if (this_df["names_code"].tolist()[row_idx] == this_appr_name_code):
                this_color = '#F7B500'
                new_rd = True
                travel_color = "Yellow"

            elif this_speed > ave_speed:
                # faster than **average** speed
                this_color = '#33A532'
                travel_color = "Green"

            else:
                this_color = '#BB1E10'
                travel_color = "Red"

        else:
            if this_speed > ave_speed:
                # faster than **average** speed
                this_color = '#33A532'
                travel_color = "Green"

            else:
                this_color = '#BB1E10'
                travel_color = "Red"

        html_color = this_color

        if constr_rd:
            text_message_short = '''
            <center>
            <p style="color:{html_c};">
            <b>UNDER CONSTRUCTION</b>
            </p>
            </center>
            <hr>
            <center>
            <p><b>Date:</b> {date_rec} <b>|</b> <b>Time:</b> {t_rec}
            </p>
            </center>
            <hr>
            <p>You have chosen this road to be <b>under construction</b>,
            so it is not in operation.
            </p>
            <p><b>Historical Average Speed:</b> {ave_s:.2f} mph
            </p>
            <hr>
            <p><b>TrafficLink Name:</b> {lk_nm}
            </p>
            <p><b>Borough:</b> {bor}
            </p>
            '''.format(lk_nm = this_df["names"].tolist()[row_idx],
                       bor = correspond_boro,
                       ave_s = ave_speed,
                       date_rec = this_df["time_s"].tolist()[row_idx].split("T")[0],
                       t_rec = this_df["time_s"].tolist()[row_idx].split("T")[1],
                       html_c = html_color)
            
            temp_iframe = folium.IFrame(text_message_short)
            msg_popup = folium.Popup(temp_iframe,
                                     min_width=300,
                                     max_width=300,
                                     min_height=500,
                                     max_height=500)

            folium.Marker(
                location = start_pt,
                popup = msg_popup,
                icon = folium.Icon(color="orange", prefix='fa', icon="exclamation-triangle"),
                tooltip = "About the proposed road construction!"
            ).add_to(this_f_map)
            
            row_idx += 1
            continue

        elif new_rd:

            text_message_short = '''
            <hr>
            <center>
            <p align=center, style="color:{html_c};">
            <b>NEW ROAD</b>
            </p>
            </center>
            <hr>
            <center>
            <p align=center><b>Date:</b> {date_rec} <b>|</b> <b>Time:</b> {t_rec}
            </p>
            </center>
            <hr>
            <p align=center>This is a <b>new road</b> that you have proposed,
            thus yet to be constructed.
            </p>
            <hr>
            <p align=center><b>Predicted Speed at This Time:</b> {speed:.2f} mph
            </p>
            <p align=center><b>Predicted Travel Time:</b> {tr_t:.2f} minutes.
            </p>
            <hr>
            <p align=center><b>TrafficLink Name:</b> {lk_nm}
            </p>
            <p align=center><b>Borough:</b> {bor}
            </p>
            <hr>
            '''.format(lk_nm = this_df["names"].tolist()[row_idx],
                       bor = correspond_boro,
                       date_rec = this_df["time_s"].tolist()[row_idx].split("T")[0],
                       t_rec = this_df["time_s"].tolist()[row_idx].split("T")[1],
                       tr_t = float(this_df["pred_tr_time"].tolist()[row_idx]) / 60,
                       speed = this_speed,
                       html_c = html_color)
            
            display(HTML(text_message_short))
            break

        else:
            text_message_short = '''
            <center>
            <p style="color:{html_c};">
            <b>TRAVEL COLOR: {clr}</b>
            </p>
            </center>
            <hr>
            <center>
            <p><b>Date:</b> {date_rec} <b>|</b> <b>Time:</b> {t_rec}
            </p>
            </center>
            <hr>
            <p><b>Speed at This Time:</b> {speed:.2f} mph
            </p>
            <p><b>Travel Time:</b> {tr_t:.2f} minutes.
            </p>
            <p><b>Average Speed:</b> {ave_s:.2f} mph
            </p>
            <hr>
            <p><b>TrafficLink Name:</b> {lk_nm}
            </p>
            <p><b>Borough:</b> {bor}
            </p>
            '''.format(lk_nm = this_df["names"].tolist()[row_idx],
                       bor = correspond_boro,
                       date_rec = this_df["time_s"].tolist()[row_idx].split("T")[0],
                       t_rec = this_df["time_s"].tolist()[row_idx].split("T")[1],
                       tr_t = float(this_df["pred_tr_time"].tolist()[row_idx]) / 60,
                       speed = this_speed,
                       ave_s = ave_speed,
                       html_c = html_color,
                       clr = travel_color.upper())

        # decide which message to use here:
        temp_iframe = folium.IFrame(text_message_short)
        msg_popup = folium.Popup(temp_iframe,
                                 min_width=300,
                                 max_width=300,
                                 min_height=500,
                                 max_height=500)

        folium.Marker(
            location = start_pt,
            popup = msg_popup,
            icon = folium.Icon(color="blue", icon="info-sign"),
            tooltip = "Click here to see more meta-info about this starting location!"
        ).add_to(this_f_map)

        row_idx += 1

    print("Proposed times is:", Times.strftime("Date: %Y-%m-%d, Time: %H:%M:%S"))
    print("Proposed modification is:", Modification)

    display(this_f_map)
    # return this_f_map

    #this_f_map.save("{}_map.html".format(Link_Name))

In [142]:
link_widget_3 = wg.Dropdown(description = 'Traffic Link name:')

boros_widget_3 = wg.ToggleButtons(
    options = all_boros,
    description = 'Borough:',
    value = None,
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    icons = ['road'] * 5
)

modif = wg.ToggleButtons(
    options = ["None", "Construction", "New Road", "New Multi-Road"],
    description = 'Modification:',
    value = None,
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    icons = ['cogs'] * 5
)

st_input = wg.Text(
    value = '80 Lafayette Street, New York, NY',
    placeholder = 'Enter starting point address',
    description = 'Starting Point:',
    disabled = False
)

end_input = wg.Text(
    value = 'Washington Square Park, New York, NY',
    placeholder = 'Enter ending point address',
    description = 'Ending Point:',
    disabled = False
)

addresses = wg.Textarea(
    placeholder = 'Type all point addresses, one per line (separate by enter/return)',
    description = 'Multi-Road:',
    disabled = False
)

e_read = wg.Checkbox(
    value = False,
    description = ''' '''
)

e_about = wg.HTML(value = '''
<p style="color:red;">By checking the checkbox above, I affirm under penalty of perjury that 
I have thoroughly read, understood, and fully acknowledged 
the <b><u><a href="https://github.com/tx542/nyc_open_data/blob/main/Project%20Notice%20of%20Objectives%20and%20Limitations.ipynb" target="_blank"><i>Notice of Project Objectives and Limitations</i></a></u></b>, and pledge, to the best extent of my knowledge, that 
I will not use this Simulator, which has been developed for artistic illustration purposes, 
for unethical extrapolation purposes.</p>''', description = " ")


min_time = pytz.timezone('America/New_York').localize(sorted([pd.to_datetime(item) for item in list(MyTraffic.pol_df['time_s'].unique())])[0])
min_t_str = min_time.strftime("%Y-%m-%dT%H:%M:%S")
min_day = min_time.strftime("%Y-%m-%d")
min_hour = int(min_time.strftime("%H"))

max_time = pytz.timezone('America/New_York').localize(sorted([pd.to_datetime(item) for item in list(MyTraffic.pol_df['time_s'].unique())])[-1])
max_t_str = max_time.strftime("%Y-%m-%dT%H:%M:%S")
max_day = max_time.strftime("%Y-%m-%d")
max_hour = int(max_time.strftime("%H"))

t_day_input = wg.Dropdown(
    options = list(set([min_day, max_day])),
    value = list(set([min_day, max_day]))[0],
    description = 'Day:',
    disabled = False
)

t_hour_input = wg.IntSlider(
    min = min_hour,
    max = max_hour,
    step = 1,
    description = 'Hour:',
    disabled = False
)

t_min_input = wg.IntSlider(
    min = 0,
    max = 59,
    step = 1,
    description = 'Minute:',
    disabled = False
)

t_sec_input = wg.IntSlider(
    min = 0,
    max = 59,
    step = 1,
    description = 'Second:',
    disabled = False
)

time_info = wg.HTML(value = 'You have not yet chosen a time...', description = ' ')
time_info0 = wg.HTML(value = '''
Please also <b>choose a time for your proposed simulation</b>, noting that the permitted time interval 
is based on that of the available sample (data points) collected, to avoid extrapolation/over-generalization:'''
                   , description = ' ')
mod_info = wg.HTML(value = '''
Please start by <b>choosing a proposed modification method first</b> (before anything else):'''
                   , description = ' ')

# Define a function that updates the content of link based on what we select for boros
def update3(*args):
    if boros_widget_3.value != None:
        link_widget_3.options = sorted(list(MyTraffic.pol_df[MyTraffic.pol_df["boro"].apply(str.lower) == boros_widget_3.value.lower()]['names'].unique()))

boros_widget_3.observe(update3)
link_widget_3.observe(update3)

def irrelavance(*args):
    if modif.value == "None":
        boros_widget_3.disabled = False # can choose boro only
        boros_widget_3.layout.visibility = 'visible'
        
        link_widget_3.disabled = True
        link_widget_3.layout.visibility = 'hidden'
        
        st_input.disabled = True
        st_input.layout.visibility = 'hidden'
        
        end_input.disabled = True
        end_input.layout.visibility = 'hidden'
        
        addresses.disabled = True 
        addresses.layout.visibility = 'hidden'
        
    elif modif.value == "Construction":
        boros_widget_3.disabled = False # can choose boro
        boros_widget_3.layout.visibility = 'visible'
        
        link_widget_3.disabled = False # and choose street link based on boro
        link_widget_3.layout.visibility = 'visible'
        
        st_input.disabled = True
        st_input.layout.visibility = 'hidden'
        
        end_input.disabled = True
        end_input.layout.visibility = 'hidden'
        
        addresses.disabled = True 
        addresses.layout.visibility = 'hidden'
        
    elif modif.value == "New Road":
        boros_widget_3.disabled = True
        boros_widget_3.layout.visibility = 'hidden'
        
        link_widget_3.disabled = True
        link_widget_3.layout.visibility = 'hidden'
        
        # only start-end bi-point input allowed
        st_input.disabled = False
        st_input.layout.visibility = 'visible'
        
        end_input.disabled = False
        end_input.layout.visibility = 'visible'
        
        addresses.disabled = True 
        addresses.layout.visibility = 'hidden'
        
    elif modif.value == "New Multi-Road":
        boros_widget_3.disabled = True
        boros_widget_3.layout.visibility = 'hidden'
        
        link_widget_3.disabled = True
        link_widget_3.layout.visibility = 'hidden'
        
        st_input.disabled = True
        st_input.layout.visibility = 'hidden'
        
        end_input.disabled = True
        end_input.layout.visibility = 'hidden'
        
        # only multi-line address input allowed
        addresses.disabled = False 
        addresses.layout.visibility = 'visible'
        
modif.observe(irrelavance)
boros_widget_3.observe(irrelavance)
link_widget_3.observe(irrelavance)
st_input.observe(irrelavance)
end_input.observe(irrelavance)
addresses.observe(irrelavance)
        
def update_time(*args):
    time_info.value = 'You have chosen <b>the day {}</b>, and <b>the time {:02d}:{:02d}:{:02d}</b>'.format(t_day_input.value,
    t_hour_input.value, t_min_input.value, t_sec_input.value)

time_info.observe(update_time)
t_day_input.observe(update_time)
t_hour_input.observe(update_time)
t_min_input.observe(update_time)
t_sec_input.observe(update_time)


In [143]:
this_interact_manual = wg.interact_manual.options(manual_name = "Run Simulation!")


#Times = wg.DatetimePicker(
#                value = min_time,
#                min = min_time,
#                max = max_time,
#                description='Pick a Time',
#                disabled=False),

this_interact_manual(comp_model2,
            Mod_label = mod_info,
            learn_more = wg.Checkbox(value = False,
                                     description = 'Learn More ("Run Simulation!")',
                                     disabled=False,
                                     indent=True),
            Modification = modif,
            time_infotext = time_info0,
            Borough = boros_widget_3,
            Link_Name = link_widget_3,
            u_day = t_day_input,
            u_hour = t_hour_input,
            u_min = t_min_input,
            u_sec = t_sec_input,
            t_info = time_info,
            start_str = st_input,
            end_str = end_input,
            all_address = addresses,
            ethics = e_read,
            ethics_about = e_about
           );

interactive(children=(HTML(value='\n<h1 align=center><font size=5>NYC Traffic Open Data: Planning Proposal Sim…

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.